In [ ]:
import os
import json
import requests 
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from IPython.display import Markdown, display
from concurrent.futures import ThreadPoolExecutor
import utils as ut

In [ ]:
# define the constants 
MAX_THREAD = 4
AI_MODELS = ["openai", "anthropic", "ollama", "deepseek"]

In [ ]:
# define the question prompt
request = "Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. "
request += "Answer only with the question, no explanation."
messages = [{"role": "user", "content": request}]

In [ ]:
# generate question 
question = ut.ask_llm("openai", messages=messages)

# re-define the messages using the question to send to the LLMs 
messages = [{"role": "user", "content": question}] 

In [ ]:
# inspect the question 
question

In [ ]:

# send question to llms 
with ThreadPoolExecutor(max_workers=MAX_THREAD) as executor:
    responses = executor.map(ut.ask_llm, AI_MODELS, [messages] * len(AI_MODELS))


In [ ]:
# join all responses 
all_reponses = "/n".join([f"# Response from model {index} \n\n {response}" for index, response in enumerate(responses, start=1)])

In [ ]:
# define agggregator prompt 
aggregator_prompt = f"""
You are a helpful assistant that aggregates the responses from a number of LLMs.

You will be given a list of responses from the LLMs.

You will need to aggregate the responses into a single response.

Each model has been given this question:
{question}

You will need to return the aggregated response in Markdown format,
making explicit where each part/section of the aggregated response comes from at the end of the response.

Here are the responses from each competitor:

{all_reponses}

""" 

In [ ]:
aggregator_messages = [{"role": "user", "content": aggregator_prompt}]

In [ ]:
# aggregate the responses 

aggregated_response = ut.ask_llm("openai", messages=aggregator_messages)

display(Markdown(aggregated_response))